In [109]:
import cvxpy as cp, numpy as np

In [110]:
def makeA2(x,y):
    u = x.reshape((-1,1))
    v = y.reshape((-1,1))
    return (u*v.T+v*u.T) / 2.  # ip(u,v)

makeA1 = lambda x: makeA2(x,x)

In [111]:
# Paramaters

N = 6     # no. iterations
L = 1   # h-smoothness constant
lam = 1/L # step-size
R = 1     # initial radius
# sig = 1   # strong-convexity paramater

In [112]:
dimM = 3*N + 5
dimF = dimG = dimH = N + 2
nbPts = N + 2

In [113]:
# encoding vectors 

x = np.zeros((nbPts, dimM))  # iterates x^*, x_0, x_1, \dots, x_N
df = np.zeros((nbPts, dimM)) # \nabla f
dh = np.zeros((nbPts, dimM)) # \nabla h
dg = np.zeros((nbPts, dimM)) # normal cone to constraints
f = np.zeros((nbPts, dimF))  # function values

x[1:,:nbPts-1] = np.eye(nbPts-1)
df[:,nbPts-1:2*nbPts-1] = np.eye(nbPts)
dg[1:,-nbPts:-1] = np.eye(nbPts-1)
dg[0,:] = -df[0,:]
dh[1,-1] = 1

f[1:,:nbPts-1] = np.eye(nbPts-1)
h = f.copy() # kernel values

In [114]:
# encoding the NoLips algorithm

for i in range(N):
    dh[i+2,:] = dh[i+1,:] - lam * (df[i+1,:] + dg[i+1,:])

### Essentialy the PEP is the same as that of NoLips, except here we include the Pythag thm for Bregman Projection.

$$\langle \nabla h(x_{k+1})  - \nabla h(x_k) + \lambda\nabla f(x_k), x - x_{k+1}\rangle \ge 0 \quad \forall x\in C$$

In [115]:
M = cp.Variable((dimM, dimM), PSD=True)
F = cp.Variable((dimF, 1))
H = cp.Variable((dimH, 1))

# initial radius constraints
constraints = {'radius': (h[0,:] - h[1,:])@H - dh[1,:] @ M @ (x[0,:] - x[1,:]) <= R} # D_h(x^*, x_0) 

# convexity constraints
for i in range(nbPts):
    for j in range(nbPts):
        if i != j:
            # convexity of f
            constraints[f'f conv @ ({i},{j})'] = (f[j,:] - f[i,:])@F + df[j,:] @ M @ (x[i,:] - x[j,:]) <= 0
            
            # convexity of g
#             constraints[f'g conv @ ({i},{j})'] = (g[j,:] - g[i,:])@G + dg[j,:] @ M @ (x[i,:] - x[j,:]) <= 0
            
            # strong-convexity of h       
            
            # convexity of Lh - f
            constraints[f'Lh - f conv @ ({i},{j})'] = L*(h[j,:] - h[i,:])@H - (f[j,:] - f[i,:])@F + (L*dh[j,:] - df[j,:]) @ M @ (x[i,:] - x[j,:]) <= 0
        
        if 0 < i < nbPts-1 and i+1!=j:
            # projection ineq (Pythagorean)
            constraints[f'Pythag @ ({i}, {j})'] = (dh[i+1,:] - dh[i,:] + lam*df[i,:]) @ M @ (x[j,:] - x[i+1,:]) >= 0

In [116]:
# objective
obj = cp.Maximize((f[-1,:] - f[0,:])@F)
prob = cp.Problem(obj,constraints.values())
prob.solve(solver=cp.MOSEK)

0.16666663136077708

In [117]:
print('PEP value: {:.4f}'.format(obj.value))
print('Theo value: {:.4f}'.format(L*R/N))

for name,c in constraints.items():
    if not np.isclose(c.dual_value, 0, atol=1e-4):
        print('\t{}: {:.4f}'.format(name, float(c.dual_value)))

PEP value: 0.1667
Theo value: 0.1667
	radius: 0.1667
	f conv @ (0,1): 0.1667
	f conv @ (0,2): 0.1667
	f conv @ (0,3): 0.1667
	f conv @ (0,4): 0.1667
	f conv @ (0,5): 0.1667
	f conv @ (0,6): 0.1667
	f conv @ (0,7): 0.1667
	Lh - f conv @ (0,7): 0.1667
	Pythag @ (1, 0): 0.1667
	Pythag @ (2, 0): 0.1667
	Lh - f conv @ (2,1): 0.1667
	Pythag @ (2, 2): 0.0679
	f conv @ (2,3): 0.0679
	Lh - f conv @ (2,3): 0.0272
	f conv @ (2,4): 0.0407
	Lh - f conv @ (2,4): 0.0131
	f conv @ (2,5): 0.0277
	Lh - f conv @ (2,5): 0.0086
	f conv @ (2,6): 0.0191
	Lh - f conv @ (2,6): 0.0078
	f conv @ (2,7): 0.0113
	Lh - f conv @ (2,7): 0.0113
	Pythag @ (3, 0): 0.1667
	Lh - f conv @ (3,2): 0.2346
	Pythag @ (3, 2): 0.0407
	Pythag @ (3, 3): 0.1070
	f conv @ (3,4): 0.1070
	Lh - f conv @ (3,4): 0.0442
	f conv @ (3,5): 0.0629
	Lh - f conv @ (3,5): 0.0218
	f conv @ (3,6): 0.0410
	Lh - f conv @ (3,6): 0.0174
	f conv @ (3,7): 0.0237
	Lh - f conv @ (3,7): 0.0237
	Pythag @ (4, 0): 0.1667
	Pythag @ (4, 2): 0.0277
	Lh - f conv @ 